# TensorFlow によるモデルの学習とマルチモデルエンドポイントでのホスティング

こちらは、TensorFlow を SageMaker 上で学習し、ひとつの推論エンドポイントに複数のモデルをデプロイする [マルチモデルエンドポイント (MME)](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/multi-model-endpoints.html#multi-model-endpoint-instance) の機能を使ってモデルをホスティングするノートブックです。このノートブックでは、SageMaker の pre-build Tensorflow コンテナを使う方法と、OSS の [Multi-Model Server (MMS)](https://github.com/awslabs/multi-model-server) を使う方法の 2通りをご紹介します。状況に合わせて適切な方法をご選択ください。2021年5月現在、マルチモデルエンドポイントは GPU インスタンスには対応していません。

[SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) は、SageMaker学習インスタンスへのスクリプトの転送を処理します。学習インスタンスでは、SageMakerのネイティブTensorFlowサポートが学習関連の環境変数を設定し、学習スクリプトを実行します。このチュートリアルでは、SageMaker Python SDKを使用して学習ジョブを起動し、学習されたモデルを展開します。
TensorFlow Training についての詳細についてはこちらの[ドキュメント](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html#train-a-model-with-tensorflow)にアクセスしてください

このノートブックでは、マルチモデルエンドポイントを実現するための 2通りの方法をご紹介しますが、それぞれの基本的な違いは以下の通りです。

- SageMaker の pre-build Tensorflow コンテナを使う方法
  - Tensorflow 2.2.2 かそれより新しいバージョンを使用する場合、かつ、推論データの前処理、後処理が不要な場合。<br>基本的にこちらの方法がシンプルでおすすめ。
- Multi-Model Server を使う方法
  - マルチモデルエンドポイントがサポートされていないバージョンの Tensorflow を使う場合、または、推論データの前処理、後処理が必要な場合。

**※本ノートブックは TensorFlow バージョン 2 以上で動作します。**

---

## コンテンツ
モデルを学習する部分は通常の SageMaker 学習ジョブで行います。学習ジョブによって作成された学習済みモデルを使ってマルチモデルエンドポイントを作成します。

1. [環境のセットアップ](#1.-環境のセットアップ)
1. [学習データの準備](#2.-学習データの準備)
1. [分散学習用のスクリプトを作成する](#3.-分散学習用のスクリプトを作成する)
1. [TensorFlow Estimator を利用して学習ジョブを作成する](#4.-TensorFlow-Estimatorを利用して学習ジョブを作成する)
1. [Pre-build Tensorflow コンテナを使ってマルチモデルエンドポイントを作成する](#5.-Pre-build-Tensorflow-コンテナを使ってマルチモデルエンドポイントを作成する)
1. [Multi-Model Server を使ってマルチモデルエンドポイントを作成する](#6.-Multi-Model-Server-を使ってマルチモデルエンドポイントを作成する)
1. [エンドポイントを削除する](#7.エンドポイントを削除する)
---


# 1. 環境のセットアップ

まずは環境のセットアップを行いましょう。

In [ ]:
import os, sagemaker, urllib
import matplotlib.pyplot as plt
import numpy as np
import boto3

from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

role = get_execution_role()
region = sagemaker_session.boto_session.region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
s3_output = sagemaker_session.default_bucket()
s3_prefix = 'tensorflow-mme'
tag = ':latest'

print(f'Current SageMaker Python SDK Version = {sagemaker.__version__}')

注） このノートブックでは SageMaker SDK が 2.19.0 以上で動作します。上記の出力結果がそれ以前のバージョンになった際は、下記のセルの#を削除（コメントアウトを解除）して実行、Jupyterカーネルを再起動し、再度上記のセルを実行し、バージョンがアップデートされたことを確認してください。カーネルが再起動されない場合は、SageMaker SDK バージョン更新が反映されません。

In [ ]:
# !pip install -U --quiet "sagemaker>=2.19.0"

# 2. 学習データの準備

MNISTデータセットは、パブリックS3バケット ``sagemaker-sample-data-<REGION>`` の下のプレフィックス ``tensorflow/mnist`` の下にロードされています。 このプレフィックスの下には4つの ``.npy`` ファイルがあります：
* ``train_data.npy``
* ``eval_data.npy``
* ``train_labels.npy``
* ``eval_labels.npy``

学習データが保存されている s3 の URI を変数に格納しておきます。

In [ ]:
training_data_uri = f's3://sagemaker-sample-data-{region}/tensorflow/mnist/'
print(training_data_uri)
!aws s3 ls {training_data_uri}

# 3. 分散学習用のスクリプトを作成する

このチュートリアルの学習スクリプトは、TensorFlowの公式の [CNN MNISTの例](https://www.tensorflow.org/tutorials/images/cnn?hl=ja) をベースに作成されました。 SageMaker から渡された `` model_dir`` パラメーターを処理するように変更しています。 これは、分散学習時のデータ共有、チェックポイント、モデルの永続保存などに使用できるS3パスです。 また、学習関連の変数を扱うために、引数をパースする関数も追加しました。

学習ジョブの最後に、学習済みモデルを環境変数 ``SM_MODEL_DIR`` に保存されているパスにエクスポートするステップを追加しました。このパスは常に ``/opt/ml/model`` をポイントします。 SageMaker は、学習の終了時にこのフォルダー内のすべてのモデル成果物をS3にアップロードするため、これは重要です。

スクリプト全体は次のとおりです。

In [ ]:
!pygmentize 'mnist.py'

# 4. TensorFlow Estimatorを利用して学習ジョブを作成する

`sagemaker.tensorflow.TensorFlow`　estimator は、スクリプトモード対応の TensorFlow コンテナの指定、学習・推論スクリプトの S3 へのアップロード、および SageMaker 学習ジョブの作成を行います。ここでいくつかの重要なパラメーターを呼び出しましょう。

* `py_version`は` 'py3'`に設定されています。レガシーモードは Python 2 のみをサポートしているため、この学習スクリプトはスクリプトモードを使用していることを示しています。Python2は間もなく廃止されますが、 `py_version` を設定することでPython 2でスクリプトモードを使用できます。`'py2'`と` script_mode`を `True`にします。

* `distributions` は、分散学習設定を構成するために使用されます。インスタンスのクラスターまたは複数の GPU をまたいで分散学習を行う場合にのみ必要です。ここでは、分散学習スキーマとしてパラメーターサーバーを使用しています。 SageMaker 学習ジョブは同種のクラスターで実行されます。 SageMaker セットアップでパラメーターサーバーのパフォーマンスを向上させるために、クラスター内のすべてのインスタンスでパラメーターサーバーを実行するため、起動するパラメーターサーバーの数を指定する必要はありません。スクリプトモードは、[Horovod](https://github.com/horovod/horovod) による分散学習もサポートしています。 `distributions` の設定方法に関する詳細なドキュメントは[こちら](https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/tensorflow#distributed-training) をご参照ください。

* 実際にモデル開発をする際はコード(ここでは `mnist.py` )にバグが混入していないか確認しながら実行することになりますが、学習インスタンスを利用すると、インスタンスの起動に時間がかかるため、学習開始コマンドを打ち込んでから 10 分後に気づいてやり直し、となってしまうことがあります。そのオーバヘッドを防止するために、ローカルモードでの学習が Sagemaker ではサポートされています。``instance_type=local``を指定するだけで、ノートブックインスタンスで学習（＝インスタンスの立ち上げ時間なしで）を試すことができます。よくやるやり方としてはコードの確認用途のため、 epoch の数やデータを減らして動くかどうかの確認を行うことが多いです。

また、Spot Instanceを用いて実行する場合は、下記のコードを `Estimator` の `train_instance_type` の次の行に追加しましょう。

```python
                             max_run = 5000, # 学習は最大で5000秒までにする設定
                             use_spot_instances = 'True',
                             max_wait = 7200 # 学習完了を待つ最大時間
```




In [ ]:
from sagemaker.tensorflow import TensorFlow


mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             instance_count=2,
                             # instance_type='local',
                             instance_type='ml.p3.2xlarge',
                             framework_version='2.2.2',
                             py_version='py37',
                             distribution={'parameter_server': {'enabled': True}},
                             hyperparameters={
                                 "epochs": 4,
                                 'batch-size':16
                             }
#                              max_run = 5000, # 学習は最大で5000秒までにする設定
#                              use_spot_instances = 'True',
#                              max_wait = 7200 # 学習完了を待つ最大時間
                            )

## ``fit`` による学習ジョブの実行

学習ジョブを開始するには、`estimator.fit（training_data_uri）` を呼び出します。

ここでは、S3 ロケーションが入力として使用されます。 `fit` は、`training` という名前のデフォルトチャネルを作成します。これは、このS3ロケーションを指します。学習スクリプトでは、 `SM_CHANNEL_TRAINING` に保存されている場所から学習データにアクセスできます。 `fit`は、他のいくつかのタイプの入力も受け入れます。詳細については、APIドキュメント[こちら](https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.EstimatorBase.fit) を参照してください。

学習が開始されると、TensorFlow コンテナは mnist.py を実行し、スクリプトの引数として　estimator から`hyperparameters` と `model_dir` を渡します。この例では、estimator 内で定義していないハイパーパラメーターは渡されず、 `model_dir` のデフォルトは `s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>` であるため、スクリプトの実行は次のようになります。
```bash
python mnist.py --model_dir s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>
```
学習が完了すると、学習ジョブは保存されたモデルを TensorFlow serving にアップロードします。

In [ ]:
mnist_estimator.fit(training_data_uri)

マルチモデルエンドポイントは、指定された S3 パスにデプロイしたい全てのモデルを保存します。このノートブックでは、先ほど実行した学習ジョブの学習済みモデルが保存されているパスをマルチモデルエンドポイント用のパスとして使用します。

以下のセルでは、学習済みモデルが保存されたパスを取得しています。

In [ ]:
import os
dirname = os.path.dirname(mnist_estimator.model_data)
dirname

複数のモデルを学習させるのは時間がかかるので、このノートブックでは先ほど学習したモデルを複製してデプロイします。

以下のセルでは、S3 に保存されている model.tar.gz を model2.tar.gz から model7.tar.gz までの 6 回複製しています。

In [ ]:
!aws s3 cp $dirname/model.tar.gz $dirname/model2.tar.gz
!aws s3 cp $dirname/model.tar.gz $dirname/model3.tar.gz
!aws s3 cp $dirname/model.tar.gz $dirname/model4.tar.gz
!aws s3 cp $dirname/model.tar.gz $dirname/model5.tar.gz
!aws s3 cp $dirname/model.tar.gz $dirname/model6.tar.gz
!aws s3 cp $dirname/model.tar.gz $dirname/model7.tar.gz

あとで推論を実行する際に使用する入力データを準備しておきます。

In [ ]:
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/eval_data.npy eval_data.npy
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/eval_labels.npy eval_labels.npy

eval_data = np.load('eval_data.npy').reshape(-1,28,28,1)
eval_labels = np.load('eval_labels.npy')

データセットから 50 枚のみ抜き出して `test_data` とします。

In [ ]:
k = 1000 # choose your favorite number from 0 to 9950
test_data = eval_data[k:k+50]
test_data

for i in range(5):
    for j in range(10):
        plt.subplot(5, 10, 10* i + j+1)
        plt.imshow(test_data[10 * i + j, :].reshape(28, 28), cmap='gray')
        plt.title(10* i + j+1)
        plt.tick_params(labelbottom=False, labelleft = False)
        plt.subplots_adjust(wspace=0.2, hspace=1)
plt.show()

# 5. Pre-build Tensorflow コンテナを使ってマルチモデルエンドポイントを作成する

まずは、SageMaker が用意している pre-build Tensorflow コンテナを使ってマルチモデルエンドポイントを作成してみましょう。Tensorflow 2.2.2 を含む新しいバージョンがマルチモデルエンドポイントに対応しています。Pre-build コンテナの一覧は [こちら](https://github.com/aws/deep-learning-containers/blob/master/available_images.md) から参照可能です。古いバージョンや、SageMaker が用意していないバージョンの Tensorflow を使いたい場合は、[6. Multi-Model Server を使ってマルチモデルエンドポイントを作成する](#6.-Multi-Model-Server-を使ってマルチモデルエンドポイントを作成する) の方法を使用してください。

まず TensorFlowModel を作成し、それを引数として MultiDataModel を作成します。

In [ ]:
from sagemaker.tensorflow.serving import TensorFlowModel
model = TensorFlowModel(role=role,
                        image_uri= '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.2.2-cpu-py37-ubuntu18.04',
                        model_data=mnist_estimator.model_data)

MultiDataModel の引数 `model_data_prefix` には、デプロイしたいモデルたちが保存されている S3 パスを指定します。

In [ ]:
from sagemaker.multidatamodel import MultiDataModel
from time import gmtime, strftime

endpoint_name = 'tensorflow-mnist-mme-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_name = endpoint_name

mme = MultiDataModel(name=model_name,
                     model_data_prefix=dirname + '/',
                     model=model,# passing our model - passes container image needed for the endpoint
                     sagemaker_session=sagemaker_session)

MultiDataModel の deploy() を使って推論エンドポイントを起動します。このエンドポイントがマルチモデルエンドポイントとなります。エンドポイントの起動には 10 分ほどかかります。セルの下にしばらく - が表示されたのち、最後に ! が表示されたらエンドポイントの起動完了です。

In [ ]:
predictor = mme.deploy(initial_instance_count=1,
                       instance_type='ml.m5.xlarge',
                       endpoint_name=endpoint_name)

エンドポイントにデプロイされているモデルの一覧を見てみましょう。先ほど複製した合計 7 つのモデルが表示されます。

In [ ]:
list(mme.list_models())

それでは、起動完了した推論エンドポイントに推論リクエストを投げて推論を実行しましょう。`instances` というラベルで入力データを指定して predict() を実行します。合わせて、`TargetModel` に使用したいモデル名を指定します。

In [ ]:
%%time

payload = {'instances': test_data.reshape(-1,28,28,1)}
predictions = predictor.predict(data=payload, initial_args={'TargetModel': 'model.tar.gz'})

推論結果を確認します。0.96 ほどの精度で MNIST 画像分類ができていることがわかります。

In [ ]:
count_true = 0
for i in range(0, 50):
    prediction = np.argmax(predictions['predictions'][i])
    label = eval_labels[i+k]
    if prediction == label:
        count_true += 1
    print(' [{}]: prediction is {}, label is {}, matched: {}'.format(i+1, prediction, label, prediction == label))
    
print('Accuracy: ', (count_true/50.0))

次に、`model2.tar.gz` を使って推論を実行します。

In [ ]:
%%time

predictions = predictor.predict(data=payload, initial_args={'TargetModel': 'model2.tar.gz'})

もう一度、`model2.tar.gz` を使って推論を実行します。先ほどの初回実行時と推論時間がどれくらい変わったでしょうか。

In [ ]:
%%time

predictions = predictor.predict(data=payload, initial_args={'TargetModel': 'model2.tar.gz'})

さらに、`model3.tar.gz` を使って推論を実行します。初回実行時は推論結果が返ってくるまでに 3秒程度かかりますが、2回目以降の呼び出しでは 100 ms ほどになっていたのではないでしょうか。これは、初回は S3 からモデルをダウンロードする必要がありますが、2回目以降はモデルがメモリにキャッシュされるためです。メモリに乗り切らないほど多数、もしくはサイズの大きいのモデルをデプロイした場合、メモリからは追い出されますが推論エンドポイントにアタッチされたストレージにモデルは保存されるため、初回推論時ほどの時間はかかりません。

In [ ]:
%%time

predictions = predictor.predict(data=payload, initial_args={'TargetModel': 'model3.tar.gz'})

## 新しいモデルのアップロード
新しくモデルを追加してみましょう。モデルの追加のためにエンドポイントの設定などを変更する必要はありません。デプロイ済みのモデルが保存されている S3 パスに新しいモデルをアップロードします。

In [ ]:
!aws s3 cp $dirname/model.tar.gz $dirname/model8.tar.gz

アップロードしたモデルがマルチモデルエンドポイントの参照先に反映されているか確認します。

In [ ]:
list(mme.list_models())

新しいモデルを使って推論を実行します。

In [ ]:
%%time

payload = {'instances': test_data.reshape(-1,28,28,1)}
predictions = predictor.predict(data=payload, initial_args={'TargetModel': 'model8.tar.gz'})

# 6. Multi-Model Server を使ってマルチモデルエンドポイントを作成する

前処理、後処理を定義したい、MME が対応していないバージョンの Tensorflow を使いたい場合、OSS の [Multi-Model Server](https://github.com/awslabs/multi-model-server) を使って MME を実現することが可能です。

まずは、推論で使用するコンテナをビルドする準備をします。

In [ ]:
!mkdir -p docker/inference

In [ ]:
%%writefile docker/inference/Dockerfile

# FROM ubuntu:18.04
FROM tensorflow/tensorflow:2.2.2-py3

# Set a docker label to advertise multi-model support on the container
LABEL com.amazonaws.sagemaker.capabilities.multi-models=true
# Set a docker label to enable container to use SAGEMAKER_BIND_TO_PORT environment variable if present
LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true

# Install necessary dependencies for MMS and SageMaker Inference Toolkit
RUN apt-get update && \
    apt-get -y install --no-install-recommends \
    build-essential \
    ca-certificates \
    openjdk-8-jdk-headless \
    python3-dev \
    curl \
    vim \
    && rm -rf /var/lib/apt/lists/* \
    && curl -O https://bootstrap.pypa.io/get-pip.py \
    && python3 get-pip.py

RUN update-alternatives --install /usr/bin/python python /usr/bin/python3 1
RUN update-alternatives --install /usr/local/bin/pip pip /usr/local/bin/pip3 1

# Install MXNet, MMS, and SageMaker Inference Toolkit to set up MMS
RUN pip3 --no-cache-dir install \
                                multi-model-server \
                                sagemaker-inference \
                                retrying

# Copy entrypoint script to the image
COPY dockerd-entrypoint.py /usr/local/bin/dockerd-entrypoint.py
RUN chmod +x /usr/local/bin/dockerd-entrypoint.py

RUN mkdir -p /home/model-server/

# Copy the default custom service file to handle incoming data and inference requests
COPY model_handler.py /home/model-server/model_handler.py

# Define an entrypoint script for the docker image
ENTRYPOINT ["python", "/usr/local/bin/dockerd-entrypoint.py"]

# Define command to be passed to the entrypoint
CMD ["serve"]

In [ ]:
%%writefile docker/inference/dockerd-entrypoint.py

import subprocess
import sys
import shlex
import os
from retrying import retry
from subprocess import CalledProcessError
from sagemaker_inference import model_server

def _retry_if_error(exception):
    return isinstance(exception, CalledProcessError or OSError)

@retry(stop_max_delay=1000 * 50,
       retry_on_exception=_retry_if_error)
def _start_mms():
    # by default the number of workers per model is 1, but we can configure it through the
    # environment variable below if desired.
    # os.environ['SAGEMAKER_MODEL_SERVER_WORKERS'] = '2'
    model_server.start_model_server(handler_service='/home/model-server/model_handler.py:handle')

def main():
    if sys.argv[1] == 'serve':
        _start_mms()
    else:
        subprocess.check_call(shlex.split(' '.join(sys.argv[1:])))

    # prevent docker exit
    subprocess.call(['tail', '-f', '/dev/null'])
    
main()

推論リクエストを処理する部分を model_handler.py で定義します。`initialize()` で学習済みモデルをロードし、`preprocess()` でデータの前処理、`inference()` で推論実行、`postprocess()` で推論結果の後処理をします。

In [ ]:
%%writefile docker/inference/model_handler.py

from collections import namedtuple
import glob
import json
import logging
import os
import re

import numpy as np
import tensorflow as tf
import os,json,argparse
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy

class ModelHandler(object):
    """
    A sample Model handler implementation.
    """

    def __init__(self):
        self.initialized = False
        self.model = None
        self.shapes = None

    def initialize(self, context):
        """
        Initialize model. This will be called during model loading time
        :param context: Initial context contains model server system properties.
        :return:
        """
        self.initialized = True
        properties = context.system_properties
        # Contains the url parameter passed to the load request
        model_dir = properties.get("model_dir") 
         
        # Load  model
        try:
            self.model = tf.keras.models.load_model(os.path.join(model_dir, '000000001'))
            
        except (RuntimeError) as memerr:
            if re.search('Failed to allocate (.*) Memory', str(memerr), re.IGNORECASE):
                logging.error("Memory allocation exception: {}".format(memerr))
                raise MemoryError
            raise           

    def preprocess(self, request):
        """
        Transform raw input into model input data.
        :param request: list of raw requests
        :return: list of preprocessed model input data
        """
        # Take the input data and pre-process it make it inference ready

        payload = request[0]['body']
        
        data =  np.frombuffer(payload, dtype=np.float32).reshape(-1,28,28,1)
        return data

    def inference(self, model_input):
        """
        Internal inference methods
        :param model_input: transformed model input data list
        :return: list of inference output in NDArray
        """
        prediction = self.model.predict(model_input)
        return prediction

    def postprocess(self, inference_output):
        """
        Return predict result in as list.
        :param inference_output: list of inference output
        :return: list of predict results
        """
        print('======inference=======')
        return [str(inference_output.tolist())]
        
    def handle(self, data, context):
        """
        Call preprocess, inference and post-process functions
        :param data: input data
        :param context: mms context
        """
        
        model_input = self.preprocess(data)
        model_out = self.inference(model_input)
        return self.postprocess(model_out)

_service = ModelHandler()


def handle(data, context):
    if not _service.initialized:
        _service.initialize(context)

    if data is None:
        return None

    return _service.handle(data, context)

作成したファイルを使って Docker イメージをビルドし、Amazon ECR に push します。

In [ ]:
ecr_repository_inference = 'tensorflow-mme'
uri_suffix = 'amazonaws.com'
inference_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository_inference + tag)

# Create ECR repository and push docker image
!docker build -t $ecr_repository_inference docker/inference
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository_inference
!docker tag {ecr_repository_inference + tag} $inference_repository_uri
!docker push $inference_repository_uri

push したコンテナイメージを使って `create_model` を実行します。コンテナイメージを指定する際に、デプロイしたいモデルたちが保存されている S3 パスも指定します。

In [ ]:
from time import gmtime, strftime

model_name = 'tf-MultiModelModel-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

container = {
    'Image': inference_repository_uri,
    'ModelDataUrl': dirname + '/',
    'Mode': 'MultiModel'
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

`create_model` で作成したモデルを使って `create_endpoint_config` を実行してエンドポイント設定を作成します。

In [ ]:
endpoint_config_name = 'tf-MultiModelEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': 'ml.m5.xlarge',
        'InitialInstanceCount': 2,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

作成したエンドポイント設定を使って推論エンドポイントを起動します。このエンドポイントがマルチモデルエンドポイントとなります。エンドポイントの起動には 10 分ほどかかります。

In [ ]:
import time

endpoint_name_mms = 'tensorflow-mnist-mme-mms-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name_mms)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name_mms,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name_mms)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name_mms))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name_mms)

推論エンドポイントの起動が完了したら、`invoke_endpoint()` を使って推論を実行します。まずは `model.tar.gz` を使って推論を実行します。

In [ ]:
%%time

import json

predictions = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name_mms,
    ContentType='application/x-npy',
    TargetModel='model.tar.gz', # this is the rest of the S3 path where the model artifacts are located
    Body=test_data.reshape(-1,28,28,1).tobytes())

推論エンドポイントからは str として結果が返ってくるので、それを numpy に変換します。

In [ ]:
pred_str = predictions['Body'].read().decode('utf-8')
pred = np.array(json.loads(pred_str))

推論結果をラベルデータと比較します。問題なく推論できていそうです。

In [ ]:
count_true = 0
for i in range(0, 50):
    prediction = np.argmax(pred[i])
    label = eval_labels[i+k]
    if prediction == label:
        count_true += 1
    print(' [{}]: prediction is {}, label is {}, matched: {}'.format(i+1, prediction, label, prediction == label))
    
print('Accuracy: ', (count_true/50.0))

もう一度、`model.tar.gz` で推論を実行してみます。推論にかかる時間はどれくらい変わったでしょうか？

In [ ]:
%%time

import json

predictions = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name_mms,
    ContentType='application/x-npy',
    TargetModel='model.tar.gz', # this is the rest of the S3 path where the model artifacts are located
    Body=test_data.reshape(-1,28,28,1).tobytes())

今度は `model2.tar.gz` を実行してみます。すぐ上のセルに表示された実行時間と比べてどうでしょうか。

初回実行時は推論結果が返ってくるまでに 6 秒程度かかりますが、2回目以降の呼び出しでは 100 ms ほどになっていたのではないでしょうか。これは、初回は S3 からモデルをダウンロードする必要がありますが、2回目以降はモデルがメモリにキャッシュされるためです。メモリに乗り切らないほど多数、もしくはサイズの大きいのモデルをデプロイした場合、メモリからは追い出されますが推論エンドポイントにアタッチされたストレージにモデルは保存されるため、初回推論時ほどの時間はかかりません。

In [ ]:
%%time

import json

predictions = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name_mms,
    ContentType='application/x-npy',
    TargetModel='model2.tar.gz', # this is the rest of the S3 path where the model artifacts are located
    Body=test_data.reshape(-1,28,28,1).tobytes())

## 新しいモデルのアップロード
新しくモデルを追加してみましょう。モデルの追加のためにエンドポイントの設定などを変更する必要はありません。デプロイ済みのモデルが保存されている S3 パスに新しいモデルをアップロードします。

In [ ]:
!aws s3 cp $dirname/model.tar.gz $dirname/model9.tar.gz

アップロードしたモデルを使って推論を実行します。

In [ ]:
%%time

import json

predictions = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name_mms,
    ContentType='application/x-npy',
    TargetModel='model9.tar.gz', # this is the rest of the S3 path where the model artifacts are located
    Body=test_data.reshape(-1,28,28,1).tobytes())

# 7.エンドポイントを削除する

推論用エンドポイントは停止されるまで課金が発生します。そのため。不要になったエンドポイントはすぐに削除することをおすすめします。以下のコードでエンドポイントが削除されます。AWS コンソールの左側のメニューから「エンドポイント」をクリックし、停止したいエンドポイントを選択して削除することも可能です。

こちらは、pre-build コンテナを使用して作成したエンドポイントを削除するコードです。

In [ ]:
predictor.delete_endpoint()

こちらは、カスタムコンテナと Multi-Model Server を使って作成したエンドポイントを削除するコードです。

In [ ]:
response = sm_client.delete_endpoint(
    EndpointName=endpoint_name_mms
)